In [ ]:
import numpy as np
import pandas as pd
import os

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import pywt
from PIL import Image  
import PIL  
from skimage.io import imsave, imread

In [ ]:
from dask import bag, diagnostics 
#import hvplot.pandas
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
import shutil

import cv2
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay, zero_one_loss, classification_report
import seaborn as sns
from tensorflow import keras
import matplotlib.cm as cm
from IPython.display import Image, display

from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten
from keras.layers import Input
from keras.layers import Convolution2D , concatenate ,Conv2D,Dense
from keras.layers import Activation,Dropout, GlobalAveragePooling2D,MaxPooling2D
from keras.models import Model
from keras.optimizers import Adam ,SGD,RMSprop

In [ ]:
import math
from keras.layers import MaxPool2D, concatenate, AveragePooling2D
import tensorflow as tf
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ModelCheckpoint
import h5py
from keras.layers import Concatenate
import keras.backend as K
import keras
from keras.utils import np_utils
from keras.callbacks import LearningRateScheduler

In [ ]:
from glob import glob 
from tensorflow.keras.layers import Lambda
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
base_path = '/content/drive/MyDrive/Sem 2/CV/Sensors_data/'

# **Humidity Data**

In [ ]:
data_path = base_path + 'Humidity/'

In [ ]:
classes = list(os.listdir(data_path))
print(classes)

['Winter', 'Summer', 'Spring', 'Autumn']


In [ ]:
import PIL.Image

def cs_images(path,img_shape, classes):
    imag = []
    label = []
    for i in classes:
        l1 = os.listdir(path+'/'+i)
        for j in l1:
            img = PIL.Image.open(path+'/'+i+'/'+j)
            img = img.resize(size = (img_shape[0], img_shape[0]))
            #img = img.convert('L')
            #img = img.reshape(50,50,1)
            #img = img.flatten()
            img = cv2.imread(path+'/'+i+'/'+j)
            img = cv2.resize(img,(img_shape[0],img_shape[1]),3)
            #img = np.array(img).reshape(img_shape)
            #img = img.reshape(img_shape)
            imag.append(img)
            label.append(classes.index(i))
            del(img)
    # %matplotlib inline 
    # plt.figure(figsize=(10, 10))
    # for i in range(20):
    #     plt.subplot(5,4,i+1)
    #     plt.imshow(imag[i])
    #     plt.axis('off')
    # plt.show()
    return np.array(imag),label

In [ ]:
IMG_SIZE = 224

In [ ]:
x,y = cs_images(data_path,(IMG_SIZE, IMG_SIZE, 1),classes)

In [ ]:
y = pd.Series(y)

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(x,y,
                                                 test_size = 0.2,
                                                 random_state = 1)

In [ ]:
x_train.shape

(273, 224, 224, 3)

In [ ]:
y_train[:10]

85     0
73     0
287    3
304    3
165    1
329    3
167    1
14     0
244    2
105    1
dtype: int64

In [ ]:
y_train.shape

(273,)

In [ ]:
x_test.shape

(69, 224, 224, 3)

In [ ]:
y_test.shape

(69,)

In [ ]:

IMAGE_SIZE = [224, 224] 

In [ ]:
#Create a VGG16 model, and removing the last layer that is classifying 1000 images. This will be replaced with images classes we have. 
#vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False) #Training with Imagenet weights
# Use this line for VGG19 network. Create a VGG19 model, and removing the last layer that is classifying 1000 images. This will be replaced with images classes we have. 


vgg = VGG19(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)
# This sets the base that the layers are not trainable. If we'd want to train the layers with custom data, these two lines can be ommitted. 
for layer in vgg.layers:
  layer.trainable = False

x = Flatten()(vgg.output) #Output obtained on vgg16 is now flattened. 
x = Dropout(0.3)(x)
x = Dense(30, activation='relu')(x)
prediction = Dense(len(classes), activation='softmax')(x) # We have 5 classes, and so, the prediction is being done on len(folders) - 5 classes
#Creating model object 
model = Model(inputs=vgg.input, outputs=prediction)
model.summary()

80134624/80134624 [==============================] - 1s 0us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                              

### SGDM

In [ ]:
opt = tf.keras.optimizers.Adagrad(
    learning_rate= 0.1,
    initial_accumulator_value=0.1,
    epsilon=1e-07,
    weight_decay=None,
    clipnorm=None,
    clipvalue=None,
    global_clipnorm=None,
    use_ema=False,
    ema_momentum=0.99,
    ema_overwrite_frequency=None,
    jit_compile=True,
    name="Adagrad",
    #**kwargs
)

In [ ]:
#opt = keras.optimizers.Adam(learning_rate=0.01)
#opt = tf.keras.optimizers.experimental.SGD(0.01, momentum=0.9)
#Compile the model 
model.compile(loss='sparse_categorical_crossentropy', 
              optimizer= opt, 
              metrics=['accuracy']) 

In [ ]:
es = [
    keras.callbacks.EarlyStopping(
        # Stop training when `val_loss` is no longer improving
        monitor='val_loss',
        # "no longer improving" being defined as "no better than 1e-2 less"
        min_delta=1e-3,
        # "no longer improving" being further defined as "for at least 2 epochs"
        patience=25,
        verbose=1)
]

In [ ]:
history = model.fit(x_train,
                    y_train,
                    epochs= 200,
                    validation_data = (x_test,y_test),
                    verbose = 1,
                    #callbacks = [es],
                    batch_size=32)

Epoch 1/200
9/9 [==============================] - 304s 34s/step - loss: 321.2348 - accuracy: 0.2930 - val_loss: 1.3872 - val_accuracy: 0.2464
Epoch 2/200
9/9 [==============================] - 303s 35s/step - loss: 1.3752 - accuracy: 0.3040 - val_loss: 1.3894 - val_accuracy: 0.2464
Epoch 3/200
9/9 [==============================] - 297s 34s/step - loss: 1.3651 - accuracy: 0.3040 - val_loss: 1.3905 - val_accuracy: 0.2464
Epoch 4/200
9/9 [==============================] - 301s 34s/step - loss: 1.3677 - accuracy: 0.3040 - val_loss: 1.3916 - val_accuracy: 0.2464
Epoch 5/200
9/9 [==============================] - 296s 34s/step - loss: 1.3675 - accuracy: 0.3040 - val_loss: 1.3922 - val_accuracy: 0.2464
Epoch 6/200
9/9 [==============================] - 296s 34s/step - loss: 1.3675 - accuracy: 0.3040 - val_loss: 1.3935 - val_accuracy: 0.2464
Epoch 7/200
9/9 [==============================] - 269s 31s/step - loss: 1.3664 - accuracy: 0.2747 - val_loss: 1.3956 - val_accuracy: 0.2464
Epoch 8/200

In [ ]:
# loss
plt.plot(history.history['loss'], label='train loss')
plt.plot(history.history['val_loss'], label='val loss')
plt.legend()
plt.show()
 
 # accuracies
plt.plot(history.history['accuracy'], label='train acc')
plt.plot(history.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()

In [ ]:
from tensorflow.keras.models import load_model
model.save('/content/drive/MyDrive/Sem 2/CV/Saved_models/sensors_humidity_vgg19.h5')

In [ ]:
y_pred = np.argmax(model.predict(x_test),axis=1)

In [ ]:
y_pred[0:5]

In [ ]:
list(y_test[0:5])

In [ ]:
model.evaluate(x_test,y_test)

In [ ]:
print(classes)
# print(y_test[:25], '\n')
# print(y_pred[:25], '\n')

# predict = []
# for i in range(3000): #len(y_pred)
#   predict.append(np.argmax(y_pred[i]))

# print(predict[:25], '\n')

cm = confusion_matrix(y_test, y_pred)
print(cm, '\n')

zol = zero_one_loss(y_test, y_pred, normalize = False)
print(zol, '\n')

cr = classification_report(y_test, y_pred)
print(cr)

In [ ]:
plt.figure(figsize = (5,5))
ConfusionMatrixDisplay(cm).plot()
plt.show()